In [2]:
from PIL import Image, ImageOps
from IPython.display import display_png
import os
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.python import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Input, MaxPool2D, UpSampling2D, Lambda
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator
import tensorflow as tf
from tensorflow import keras

import json
from tensorflow.python.keras.layers import Conv2DTranspose
import glob

In [3]:
config = json.load(open('./settings.json'))
DATASET_PATH = config['dataset_path']

やりたいこと

train,test,validで分割する
分割した中で、バッチサイズごとにyieldする
そのパスを開いて、教師データ、トレーニングデータにチャンネル数を変える


In [5]:
Left_RGB = glob.glob(os.path.join(DATASET_PATH['Left_RGB'], '*png'))
Right_disparity = glob.glob(os.path.join(DATASET_PATH['Right_disparity'], '*png'))
Left_disparity = glob.glob(os.path.join(DATASET_PATH['Left_disparity'], '*png'))

Right_RGB = glob.glob(os.path.join(DATASET_PATH['Right_RGB'], '*png'))

def train_valid_test_splits(img_total_num, train_rate=0.8, valid_rate=0.1, test_rate=0.1):
    data_array = list(range(img_total_num))
    tr = math.floor(img_total_num * train_rate)
    vl = math.floor(img_total_num * (train_rate + valid_rate))

    random.shuffle(data_array)
    train_list = data_array[:tr]
    valid_list = data_array[tr:vl]
    test_list = data_array[vl:]

    return train_list, valid_list, test_list


def get_5channel_img_and_teach_img_from_data_list(batch_list, Left_RGB=Left_RGB, Right_RGB=Right_RGB, Left_disparity=Left_disparity,
                                                  Right_disparity=Right_disparity, INPUT_SIZE=(128, 128)):
    teach_img_list = []
    input_5_channel_img_list = []
    for i in batch_list:
        L_RGB = img_to_array(load_img(Left_RGB[i], target_size=INPUT_SIZE)).astype(np.uint8)
        L_DIS = img_to_array(load_img(Left_disparity[i], grayscale=True, target_size=INPUT_SIZE)).astype(np.uint8)
        R_DIS = img_to_array(load_img(Right_disparity[i], grayscale=True, target_size=INPUT_SIZE)).astype(np.uint8)

        input_5_channel_img = np.concatenate((L_RGB, L_DIS, R_DIS), 2).astype(np.uint8)
        input_5_channel_img_list.append(input_5_channel_img)

        teach_img = img_to_array(load_img(Right_RGB[i], target_size=INPUT_SIZE)).astype(np.uint8)
        teach_img_list.append(teach_img)
# 4次元テンソルに変換している
    print('shape↓')
    print(np.stack(teach_img_list).shape, np.stack(input_5_channel_img_list).shape)
    return np.stack(input_5_channel_img_list), np.stack(teach_img_list)


def generator_with_preprocessing(data_list, batch_size, shuffle=False):
    while True:
        if shuffle:
            np.random.shuffle(data_list)
        for i in range(0, len(data_list), batch_size):
            batch_list = data_list[i:i + batch_size]
            batch_5, batch_teach = get_5channel_img_and_teach_img_from_data_list(data_list)

            yield(batch_5, batch_teach)


In [6]:
len(Left_RGB)

336

In [8]:
train_list, valid_list, test_list = train_valid_test_splits(len(Left_RGB))


In [12]:
batch_size=4

In [13]:
train_gen = generator_with_preprocessing(train_list, batch_size)